api doc : https://www.coingecko.com/api/documentations/v3

In [61]:
import numpy as np
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import time
import datetime
import pandas as pd
from pathlib import Path
import os

from statsmodels.tsa.stattools import adfuller #Augmented Dickey-Fuller test
import matplotlib.pyplot as plot

ModuleNotFoundError: No module named 'statsmodels'

url = 'https://api.coingecko.com/api/v3/coins/bitcoin/history'
parameters = {
  'date':'12-11-2020'
}

session = Session()

url = 'https://api.coingecko.com/api/v3/coins/bitcoin/history'
parameters = {
  'date':'12-11-2020',
  'localization' : False
}

session = Session()


https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range?vs_currency=usd&from=1604188800&to=1605283978
endpoint pour une fourchette de date

In [60]:
start_date = "24/11/2020"
end_date = "25/11/2020"
start_timestamp = int(time.mktime(datetime.datetime.strptime(start_date, "%d/%m/%Y").timetuple()))
end_timestamp  = int(time.mktime(datetime.datetime.strptime(end_date, "%d/%m/%Y").timetuple()))

print(start_timestamp, end_timestamp)

1606172400 1606258800


In [48]:
currency="bitcoin"
url = 'https://api.coingecko.com/api/v3/coins/'+currency+'/market_chart/range?'
parameters = {
    'vs_currency':'usd',
    'from':start_timestamp,
    'to':end_timestamp
}

session = Session()

try:
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
    mypath = os.path.abspath('')
    path_to_save = str(mypath) + '/../data/'
    df = pd.DataFrame(data["prices"])
    file_name = "data.csv"
    df.to_csv(os.path.join(path_to_save, file_name))
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [49]:
csv_file_name = '../data/data.csv' #please put the name of the csv file you want to read
data = pd.read_csv(csv_file_name, index_col=[0],sep=',', encoding="utf-8")
data.iloc[:49] # we display the first 50 rows

,0,1
0,1606173556811,18316.102066
1,1606177044644,18273.492427
2,1606181004181,18450.612950
3,1606184361569,18390.498946
4,1606187723077,18332.293717
5,1606191561440,18349.033019
6,1606194969792,18374.112234
7,1606198029568,18303.951253
8,1606201977184,18407.624007
9,1606205144691,18369.329819


In [50]:
data.columns = ['time', 'price']

In [51]:
data['time'] = data['time'].astype(str).str[:-3].astype(np.int64)

In [52]:
data

,time,price
0,1606173556,18316.102066
1,1606177044,18273.492427
2,1606181004,18450.612950
3,1606184361,18390.498946
4,1606187723,18332.293717
5,1606191561,18349.033019
6,1606194969,18374.112234
7,1606198029,18303.951253
8,1606201977,18407.624007
9,1606205144,18369.329819


In [53]:
data['time'] = pd.to_datetime(data['time'], unit='s')

In [54]:
data

,time,price
0,2020-11-23 23:19:16,18316.102066
1,2020-11-24 00:17:24,18273.492427
2,2020-11-24 01:23:24,18450.612950
3,2020-11-24 02:19:21,18390.498946
4,2020-11-24 03:15:23,18332.293717
5,2020-11-24 04:19:21,18349.033019
6,2020-11-24 05:16:09,18374.112234
7,2020-11-24 06:07:09,18303.951253
8,2020-11-24 07:12:57,18407.624007
9,2020-11-24 08:05:44,18369.329819


In [55]:
def ADFtest(serie):
    val = serie.values
    result = adfuller(val)
    print('ADF Statistic: ' + result[0]) #the more negative, the more we reject null hypothesis = stationnary
    print('p-value: ' + result[1]) #p<0.05 => reject null hypothesis = stationnary



In [56]:
def plotting(serie, title):
    graph = serie.set_index('time')
    plot.plot(graph)
    plot.xlabel('Date', fontsize=12)
    plot.ylabel('Price in USD', fontsize=12)
    plot.title(title, fontsize=15)
    plot.show()

In [58]:
plotting(data, "test")

NameError: name 'plot' is not defined